# Clipper Tutorial: Part One

This tutorial will walk you through the process of starting Clipper, creating and querying a Clipper application, and deploying models to Clipper. In the first part of the demo, you will set up Clipper and create an application without involving any machine learning, demonstrating how a frontend developer or dev-ops engineer can set up and query Clipper without having to know anything about the machine-learning models involved.

As an example, this tutorial will walk you through creating an application that labels images as either pictures of birds or planes. You will use the CIFAR-10 dataset as the source of these images.


## Download the images

As the first step in the tutorial, download the CIFAR dataset using the provided [`download_cifar.py`](download_cifar.py) script and set the dataset location.



In [ ]:
cifar_loc = "/Users/crankshaw/test/path/"
%run ./download_cifar.py $cifar_loc

# Load Cifar

The first step in building any application, using machine-learning or otherwise, is to understand the application requirements. Load the dataset into the notebook so you can examine it and better understand the dataset you will be working with. To make this easier, the `cifar_utils` library is a library of little utilities for working with CIFAR data.

In [ ]:
import cifar_utils
test_x, test_y = cifar_utils.filter_data(
    *cifar_utils.load_cifar(cifar_loc, cifar_filename="cifar_test.data", norm=True))
no_norm_x, no_norm_y = cifar_utils.filter_data(
    *cifar_utils.load_cifar(cifar_loc, cifar_filename="cifar_test.data", norm=False))

Take a look at the data you've loaded. The size and blurriness of these photos should give you a better understanding of the difficulty of the task you will ask of your machine learning models! If you'd like to see more images, increase the number of rows of images displayed -- the last argument to the function -- to a number greater than 2.

In [ ]:
%matplotlib inline
cifar_utils.show_example_images(no_norm_x, no_norm_y, 2)

# Start Clipper

Now you're ready to start Clipper! You will be using the `clipper_manager` client library to perform admninistrative commands.

If you'd like to deploy Clipper locally, you can leave the `user` and `key` variables blank and set `host="local"`. Otherwise, you can deploy Clipper remotely to a machine that you have SSH access to. Set the `user` variable to your SSH username, the `key` variable to the path to your SSH key, and the `host` variable to the remote hostname or IP address.

*Remember, Docker and Docker-Compose must be installed to before deploying Clipper.*

In [ ]:
# clipper_manager must be on your path:
import sys
import os
sys.path.append(os.path.abspath('../../management/'))
import clipper_manager as cm
# Change the username if necessary
user = ""
# Set the path to the SSH key
key = ""
# Set the SSH host
host = ""
clipper = cm.Clipper(host, user, key)

clipper.start()

Congratulations! You now have a running Clipper instance that you can start to interact with. Try listing the applications deployed to this Clipper instance:

In [ ]:
clipper.list_apps()

# Create an application

In [ ]:
app_name = "cifar_demo"
candidate_models = [
    {"model_name": "sklearn_cifar", "model_version": 1},
    {"model_name": "tf_cifar", "model_version": 1},
]

clipper.register_application(
    app_name,
    candidate_models,
    "doubles",
    "EXP4",
    slo_micros=20000)

In [ ]:
clipper.list_apps()

# Start serving

In [ ]:
import seaborn as sns
sns.set_style("whitegrid")
sns.despine()
import matplotlib as mpl
%matplotlib notebook
cifar_utils.run_serving_workload(host, app_name, test_x, test_y)

*Now that you've started an application, it's time to move on to [part two](tutorial_part_two.ipynb) of the tutorial, where you
will train and deploy some models to Clipper.*

# Cleanup

In [ ]:
clipper.cleanup()